In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import re

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [2]:
# russian opt1
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
#model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

# russian opt2
#tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
#model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

# finnish opt1
#tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
#model = AutoModel.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

# finnish opt2
#tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")
#model = AutoModel.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")

# german opt1
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

# german opt2
#tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-german-cased")
#model = AutoModel.from_pretrained("google-bert/bert-base-german-cased")

# multilingual
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [8]:
df = pd.read_csv('../data/russian/axolotl.dev.ru.tsv', sep='\t')
df

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,dev_ru_0,могильник,могильникъ,mogil'nik_UYTE5-B076I,вят. слово из Вятской губернии арх. слово из А...,"могилки, кладбище",NaN,old,old
1,dev_ru_1,могильник,могильникъ,mogil'nik_C3GhETZc5Vs,ярс. слово из Ярославской губернии крупный коч...,NaN,NaN,old,old
2,dev_ru_2,могильник,могильникъ,mogil'nik_KNs3eVn3pFY,арх. слово из Архангельской губернии походный ...,NaN,NaN,old,old
3,dev_ru_3,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,"Орёл изображает реку Халзан, его голова ― скал...",NaN,new,new
4,dev_ru_4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник,NaN,old,old
...,...,...,...,...,...,...,...,...,...
2021,dev_ru_2456,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение;...",Надо было опять торопиться. Но недавняя горячк...,NaN,new,new
2022,dev_ru_2457,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение;...","И вторая нога вошла в чулок, вернее, чулок поп...",NaN,new,new
2023,dev_ru_2458,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение;...",В порыве своей первой творческой горячки он во...,NaN,new,new
2024,dev_ru_2459,горячка,горячка,gorjachka_re-9L7XQcEo,"перен. напряженная работа, требующая немедленн...",В суматохе и горячке последних часов кого-нибу...,NaN,new,new


In [27]:
def find_sub_list(sl,l): # not used because some examples have no exact coincidence
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind+1,ind+sll # +1 for the [CLS] token

def print_nice(input_ids, index):
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    tokens[index] = '\033[91m' + tokens[index] + '\033[0m'
    print(' '.join(tokens))

def extract_letters(input_string):
    return re.sub(r'[^а-яА-Яa-zA-ZÀ-ÿ]', '', input_string)

def find_word_containing_target(sentence, target_word):
    target_word = target_word.lower()
    index = sentence.lower().find(target_word)
    if index == -1:
        return None
    start_index = sentence.rfind(" ", 0, index) + 1 if index != 0 else 0
    end_index = sentence.find(" ", index + len(target_word)) if sentence.find(" ", index + len(target_word)) != -1 else len(sentence)
    return extract_letters(sentence[start_index:end_index])

In [29]:
embeddings = []
word = ""
for index, row in df.iterrows():
    if word != "" and word != row['word']:
        print(f"{word}")
    word = row['word']          # target word
    orth = row['orth']          # usage of the target word in the example
    sense_id = row['sense_id']  # sense of the target word in the example
    gloss = row['gloss']        # definition of the target word
    example = row['example']    # usage example of the target word

    if pd.isna(example):
        print(f"{index}. No example, taking gloss [CLS] token...")
        target_index = 0
        inputs = tokenizer(gloss, return_tensors="pt")
    else:
        tokens = tokenizer.tokenize(example)

        # 1. Get the target word as it is in the example
        found_search = find_word_containing_target(example, orth)
        if found_search:
            search = found_search
        else:
            found_search = find_word_containing_target(example, word)
            if found_search:
                search = found_search
            else:
                search = orth
        search_token = tokenizer.tokenize(search)[0]

        if search_token in tokens:
            target_index = tokens.index(search_token)+1 # +1 for the [CLS] token
        elif f"##{search_token}" in tokens:
            target_index = tokens.index(f"##{search_token}")+1
        else:
            tokens = [i.lower() for i in tokenizer.tokenize(example)]
            if search_token in tokens:
                target_index = tokens.index(search_token)+1 # +1 for the [CLS] token
            elif f"##{search_token}" in tokens:
                target_index = tokens.index(f"##{search_token}")+1
            else:
                # do ALL the same process but searching the orth_token and word_token subwords within the sentence ...
                print(f"{index}. {search_token} not found, taking example [CLS] token... ({tokens})")
                target_index = 0

        inputs = tokenizer(example, return_tensors="pt")

    print_nice(inputs['input_ids'][0], target_index)

    with torch.no_grad():
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

    embeddings.append( outputs.last_hidden_state[0][target_index] )

[CLS] мог ##ил ##ки , кладбище [SEP]
1. No example, taking gloss [CLS] token...
[CLS] я ##рс . слово из Я ##росла ##вской губернии к ##рупный ко ##чка ##рни ##к , ко ##че ##гу ##рни ##к ; [SEP]
2. No example, taking gloss [CLS] token...
[CLS] ар ##х . слово из Архангельск ##ой губернии поход ##ный чем ##ода ##нчи ##к с и ##гла ##ми , ши ##лья ##ми , ни ##тка ##ми , др ##ат ##вой и ##пр . для чин ##ки од ##еж ##и и об ##ув ##и . [SEP]
[CLS] О ##рёл из ##об ##ра ##жает реку Х ##ал ##зан , его голова [UNK] с ##кал ##у на Х ##ал ##зан ##е , а ла ##пы мог ##иль ##ника [UNK] мог ##ил ##у на с ##кал ##е . [SEP]
[CLS] О ##рел мог ##иль ##ник [SEP]
[CLS] Это я , я , который в " " Г ##ам ##лет ##е " " мог ##иль ##щика играл ! " " [SEP]
6. No example, taking gloss [CLS] token...
[CLS] Ж ##ел ##то ##пе ##гий ж ##ук Si ##lp ##ha , за ##рыва ##ющий м ##ел ##кую м ##ерт ##ве ##чину . [SEP]
[CLS] В э ##кс ##позиции представлен ин ##вен ##тар ##ь из р ##ели ##кто ##вого я ##зы ##ческого ба ##лт ##ского

KeyboardInterrupt: 

In [7]:
#assert len(embeddings) == len(df), "Embeddings and dataframe have different lengths"
embeddings = torch.stack(embeddings)
embeddings.shape

torch.Size([2, 768])